# T01 · Mordred Descriptor and MACCS Fingerprints Based SVR and RF models talktorial

Authors:

* Aaryan Jaitly
* Atia Tul Wahab
* Sadia Chaudhry
* Youssef Mohamed Fathy
* Zenab Khan

## Aim of this talktorial

The main goal of this project is to use machine learning to predict LogS values (solubility). We aren't just running algorithms; we also want to understand the chemistry behind the data. We will study our dataset to make sure it makes chemical sense and is good enough to build a model on. Finally, we will compare our models to find the one that gives the most accurate and reliable results.

### Contents in _Theory_

* Data Understanding: AqSolDB data in detail
* What We Missed: The Importance of Data Provenance
* The Domain Knowledge Approach
* The Consequence of Uncurated Feature Space based on Domain Knowledge
* The Conflict Between Hyperparameters and Data Noise
* Conclusion: Model Performance vs. Chemical Reality


### Contents in _Practical_

Contents in Practical


* Data Preparation
    * Loading Data

    * Sanitization

    * Standardization

    * Descriptor Generation (Mordred and MACCS)

* Machine Learning

    * Cross-Validation

    * Support Vector Regressor (SVR)

    * Random Forest Regressor

* Model Evaluation

    * Metrics Comparison

    * Testing on Challenge Datasets 1 & 2

## Theory

Building an ML model requires more than just a dataset and a label; it requires domain expertise. To successfully predict LogS, we need a large set of molecules and encodings, but we must also critically analyze the source of our data.

We cannot blindly treat all solubility values as equal. We need to know if the experiments measured kinetic solubility (often used in high-throughput screening) or equilibrium solubility (the gold standard using Shake Flask). These methods yield different results for the same molecule. By understanding these experimental nuances, we can filter out noise and account for biases, leading to a more robust and scientifically accurate model.


### Data Understanding: AqSolDB data in detail
In this project, we utilize the AqSolDB dataset comprising around 9000 compounds from nine different public databases. This aggregation presents a major challenge due to experimental heterogeneity, as the data was not generated via a standardized protocol. Varying experimental methods across laboratories lead to variances in the 'Ground Truth' LogS values. Our modeling approach must be robust enough to discern the underlying chemical signal amidst the noise from these disparate sources.

### What We Missed: The Importance of Data Provenance
We trained machine learning models using a dataset treated as a homogeneous collection, generating molecular descriptors such as Mordred and MACCS keys. However, they did not account for the experimental heterogeneity from aggregating data sources across nine different databases, which likely capped model performance due to unaddressed 'noise'.We propose that for future iterations of the project, an intensive literature search should be conducted to annotate each data point with two essential metadata columns: (1) the specific experimental protocol used, such as Shake-flask or Potentiometric, and (2) the Solubility Type (Kinetic versus Thermodynamic/Equilibrium). We believe incorporating these experimental contexts as input features or training distinct models for each solubility type could significantly reduce variance and enhance predictive accuracy.

### The Domain Knowledge Approach
We generated a wide variety of molecular descriptors through tools like Mordred and MACCS keys. We feel merely using a large number of descriptors is insufficient for effective Quantitative Structure-Activity Relationship (QSAR) modeling; instead, it is crucial to select features that closely correlate with Aqueous Solubility (LogS), including factors such as lipophilicity, molecular weight, polarity etc. The introduction of irrelevant descriptors can lead to the Curse of Dimensionality, where regularization methods (e.g., Lasso, Ridge) can mitigate, but not completely eliminate, the risks associated with extraneous features. The importance of a rigorous pre-filtering step to ensure that only descriptors with a proven correlation to solubility are included to avoid spurious correlations and excessive training time and overall model Performance.

### The Consequence of Uncurated Feature Space on Domain Knowledge
A major limitation in our modeling strategy arose from training on all generated molecular descriptors without filtering, as we relied on the algorithm to find patterns within irrelevant data. This brute-force method is often inefficient in Chemoinformatics. Many included descriptors lack a physicochemical link to Aqueous Solubility (LogS), resulting in a diluted predictive signal and the challenge of differentiating meaningful chemical causation from random noise. A more effective approach would involve applying strict domain-based feature selection, retaining only descriptors known to affect solubility, thus simplifying the model and reducing overfitting risks.

### The Conflict Between Hyperparameters and Data Noise 
#### (Only if we use current data i.e (Noisy Data); otherwise, the aforementioned modifications may not be relevant.)
Our support vector regression (SVR) model utilized an 80-20 training-test split and was optimized through Grid Search combined with Leave-One-Out Cross-Validation (LOOCV), identifying 'best' parameters of C=10 and ϵ=0.1. Although these parameters minimized training error mathematically, A significant flaw in our modeling approach: prioritizing metrics over the chemical realities of the underlying data. Specifically, a high C value of 10 indicates a strict penalty for misclassification, encouraging the model to conform closely to outliers. Conversely, a low ϵ of 0.1 imposes a narrow fitting tolerance, compelling precise alignments to target values. This strategy, applied to the AqSolDB dataset characterized by considerable experimental noise resulted in memorizing noise rather than generalizing trends. A revised method should involve the manual adjustment of Grid Search parameters, opting for a lower C and a higher ϵ to permit greater error in fitting, thereby fostering a model that better captures the underlying solubility trends while avoiding overfitting to experimental inconsistencies.

### Conclusion: Model Performance vs. Chemical Reality
1. In our comparative analysis, the Random Forest (RF) model yielded more favorable metrics than Support Vector Regression (SVR) on the training and initial challenge datasets. While RF appeared to handle the data variance somewhat more effectively, this advantage was relative. The significant performance drop observed for both models on Challenge Dataset 2 suggests that neither algorithm was fully capable of capturing the true solubility trends for complex, unseen compounds.

2. The "Hidden" Data Problem: Experimental Heterogeneity The root cause of this generalization gap lies less in the algorithm and more in the data provenance. We treated AqSolDB as a homogeneous ground truth, but in reality, it is an aggregation of nine distinct databases with varying data densities. Crucially, we overlooked the experimental context treating Kinetic Solubility and Thermodynamic Equilibrium Solubility as identical targets. By merging these fundamentally different measurements into a single "LogS" target without distinguishing columns, we forced the model to learn from conflicting signals, effectively training on "experimental noise."

3. The Feature Space Trap Finally, our "brute force" approach to feature engineering generating all possible Mordred/MACCS descriptors diluted the predictive signal. Instead of learning from a curated set of physicochemical properties known to drive solubility (domain knowledge), the model was overwhelmed by hundreds of irrelevant features. Future iterations must prioritize quality over quantity in descriptor selection to build a model that is both accurate and chemically interpretable.

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import joblib

import rdkit
from rdkit import Chem
from rdkit.Chem.rdmolops import SanitizeMol
from rdkit.Chem.MolStandardize import rdMolStandardize
import mordred
from mordred import descriptors

In [ ]:
df = pd.read_csv("./data/AqSolDB.csv")
df.head()

In [ ]:
valid_smiles = []
valid_logs = []
for i, row in df.iterrows():
    smi = row['SMILES']
    logS = row['LogS']
    mol = Chem.MolFromSmiles(smi)
    
    if mol is not None:
        try:
            # Sanitize molecule (check valence, aromaticity, conjugation, hybridization)
            SanitizeMol(mol)
            
            # Canonicalize SMILES to have consistent representation
            clean_smi = Chem.MolToSmiles(mol, canonical=True)
            
            valid_smiles.append(clean_smi)
            valid_logs.append(logS)
        
        except Exception as e:
            # Skip molecules that fail sanitization
            continue

# Create a cleaned DataFrame
clean_df = pd.DataFrame({'SMILES': valid_smiles, 'LogS': valid_logs})

print(f"✅ Valid molecules after sanitization: {len(clean_df)} / {len(df)}")
clean_df.head()

In [ ]:
def standardize_molecule(mol):
    try:
        # Remove salts and fragments
        cleaner = rdMolStandardize.Cleanup(mol)
        # Normalize functional groups
        normalizer = rdMolStandardize.Normalize(cleaner)
        # Reionize to standard charge states
        reionizer = rdMolStandardize.Reionize(normalizer)
        return reionizer
    except:
        return mol  # return original if fails

In [ ]:
standardized_smiles = []

for smi in df['SMILES']:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        mol = standardize_molecule(mol)
        clean_smi = Chem.MolToSmiles(mol, canonical=True)
        standardized_smiles.append(clean_smi)
    else:
        standardized_smiles.append(None)

df['Standardized_SMILES'] = standardized_smiles
df = df.dropna(subset=['Standardized_SMILES']).reset_index(drop=True)

print(f"✅ Standardized dataset saved with {len(df)} molecules")
df.head()

In [ ]:
df.drop(columns="Unnamed: 0", inplace=True)

In [ ]:
df.head()

In [ ]:
df = df[['SMILES', 'Standardized_SMILES', 'LogS']]

In [ ]:
df.head()

In [ ]:
df.to_csv("./data/AqSolDB_Standardized.csv",index=False)

In [ ]:
df = pd.read_csv("data/AqSolDB_Standardized.csv")
df.head()

In [ ]:
calc = mordred.Calculator(descriptors, ignore_3D=True)
mols = [rdkit.Chem.MolFromSmiles(smi) for smi in df["Standardized_SMILES"]]
mols = calc.pandas(mols=mols)

In [ ]:
mols = mols.apply(pd.to_numeric, errors='coerce')

In [ ]:
mols.shape

In [ ]:
df_descriptors = pd.concat([df,mols],axis=1)

In [ ]:
df_descriptors.shape

In [ ]:
df_descriptors.dropna(axis=1,how="any",inplace=True)

In [ ]:
df_descriptors = df_descriptors.loc[:, (df_descriptors != 0).any(axis=0)]

In [ ]:
df_descriptors.shape

In [ ]:
df_descriptors.head()

In [ ]:
df_descriptors.to_csv("./data/AqSolDB_Standardized_Mordred.csv",index=False)

In [ ]:
plt.hist(df_descriptors["LogS"])
plt.show()

In [ ]:
sns.boxplot(df_descriptors["LogS"])

In [ ]:
(df_descriptors["LogS"] < -8).sum()

In [ ]:
sns.violinplot(df_descriptors["LogS"])

In [ ]:
(df_descriptors["LogS"] < -6.5).sum()

In [ ]:
df_descriptors = df_descriptors.select_dtypes(include='number')
df_descriptors.drop(columns="LogS", inplace=True)

corr_matrix = df_descriptors.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
threshold = 0.9
to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
df_reduced = df_descriptors.drop(columns=to_drop)

In [ ]:
df_reduced.describe()

In [ ]:
df_reduced.head()

In [ ]:
df_reduced = pd.concat([df,df_reduced],axis=1)

In [ ]:
df_reduced.shape

In [ ]:
df_reduced.to_csv("./data/AqSolDB_Standardized_Mordred_Reduced.csv")

In [ ]:
df_descriptors = pd.read_csv("./data/AqSolDB_Standardized_Mordred.csv")

In [ ]:
df_descriptors.head()

In [ ]:
# Extract features (X) and target (y)

X = df_descriptors.iloc[:, 3:]  # columns after first 5 are Mordred descriptors
y = df_descriptors['LogS']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Keep only numerics
X = X.select_dtypes(include=[np.number])  
X = X.replace([np.inf, -np.inf], np.nan)

# Keep columns with at least 80% nonNaN values
X = X.dropna(axis=1, thresh=0.8 * len(X))  
X = X.fillna(X.mean())

print(f"Features after cleaning: {X.shape}")

In [ ]:
# Split train-test da
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle = True)

print(f"Training set: {X_train.shape[0]} molecules")
print(f"Test set: {X_test.shape[0]} molecules")

In [ ]:
# Scale the data
print("Scaling data...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Solubility distribution
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(y, bins=30, alpha=0.7, color='blue')
plt.xlabel('LogS')
plt.ylabel('Count')
plt.title('Solubility Distribution')
plt.grid(True, alpha=0.3)

In [ ]:
# Train SVM model
print("Training SVM model...")

svm_model = SVR(kernel = 'rbf', C = 1.0, epsilon = 0.1 , gamma = 'scale') # Got good results with defaults

svm_model.fit(X_train_scaled, y_train)

print("SVM training completed!")

In [ ]:
# Evaluate w/ epsilon 0.1 and C = 1.0
y_pred_test = svm_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(
    "SVM's Performance on Test Set:\n"
    f"  Mean Absolute Error:      {mae:.3f}\n"
    f"  R²:                       {r2:.3f}\n"
    f"  Root Mean Squared Error:  {rmse:.3f}"
)

In [ ]:
# Visualize
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_test, alpha=0.6, s=30, color='green')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2)
plt.xlabel('Actual LogS (Test)')
plt.ylabel('Predicted LogS (Test)')
plt.title(f'SVM with Mordred Descriptors\nR² = {r2:.3f}, MAE = {mae:.3f}')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
model_data = {
    'model': svm_model,
    'scaler': scaler,
    'feature_names': X.columns.tolist()
}
joblib.dump(model_data, 'svm_solubility_model.pkl')
print("Model saved as 'svm_solubility_model.pkl'")

In [ ]:
df_mordred= pd.read_csv("./data/AqSolDB_Standardized_Mordred.csv")

In [ ]:
X = df_mordred.iloc[:, 5:]  # columns after first 5 are Mordred descriptors  # All Mordred descriptors
y = df_mordred['LogS']  # Solubility values

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Split into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

In [ ]:

print(f"Training: {X_train.shape[0]} molecules")
print(f"Testing: {X_test.shape[0]} molecules")

In [ ]:
# Create model with optimized parameters
model_rf = RandomForestRegressor(
    n_estimators=100,      # 100 trees
    max_depth=30,          # Tree depth
    min_samples_split=5,   # Min samples to split
    min_samples_leaf=2,    # Min samples at leaf
    random_state=42,
    n_jobs=-1              # Use all CPU cores
)

print("Training model...")
model_rf.fit(X_train, y_train)

print("Training complete!")


In [ ]:
# Predict on training set
y_pred_train = model_rf.predict(X_train)

# Predict on test set
y_pred_test = model_rf.predict(X_test)

print("Predictions made!")
print(f"\nFirst 5 test predictions:")
print(f"{'Actual':<12} {'Predicted':<12} {'Error':<10}")
print("-" * 35)
for i in range(5):
    actual = y_test.iloc[i]
    pred = y_pred_test[i]
    error = abs(actual - pred)
    print(f"{actual:<12.3f} {pred:<12.3f} {error:<10.3f}")

In [ ]:
print("MODEL EVALUATION")
print("=" * 60)

# Training metrics
train_r2 = r2_score(y_train, y_pred_train)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
train_mae = mean_absolute_error(y_train, y_pred_train)

# Test metrics
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_mae = mean_absolute_error(y_test, y_pred_test)

print(f"\n TRAINING SET:")
print(f"  R² Score:  {train_r2:.4f}")
print(f"  RMSE:      {train_rmse:.4f}")
print(f"  MAE:       {train_mae:.4f}")

print(f"\n TEST SET (Most Important!):")
print(f"  R² Score:  {test_r2:.4f}")
print(f"  RMSE:      {test_rmse:.4f}")
print(f"  MAE:       {test_mae:.4f}")

# Check for overfitting
gap = train_r2 - test_r2
print(f"\n OVERFITTING CHECK:")
print(f"  Train R² - Test R²: {gap:.4f}")
if gap < 0.05:
    print(f"  Good generalization!")
elif gap < 0.15:
    print(f"  Slight overfitting")
else:
    print(f"   Significant overfitting")


In [ ]:
print("visualizations...")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Actual vs Predicted
axes[0].scatter(y_test, y_pred_test, alpha=0.6, edgecolors='k', s=30)
min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual LogS', fontweight='bold')
axes[0].set_ylabel('Predicted LogS', fontweight='bold')
axes[0].set_title(f'Test Set: Actual vs Predicted\nR² = {test_r2:.4f}', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Residuals
residuals = y_test - y_pred_test
axes[1].scatter(y_pred_test, residuals, alpha=0.6, edgecolors='k', s=30)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted LogS', fontweight='bold')
axes[1].set_ylabel('Residuals (Error)', fontweight='bold')
axes[1].set_title(f'Residual Distribution\nMAE = {test_mae:.4f}', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
print("Saving trained model...")

# Save model
joblib.dump(model_rf, 'random_forest_model.pkl')

print("Model saved as 'random_forest_model.pkl'")

In [ ]:
filename = "challenge_data_1.csv"
standardized_filename = "standardized_"+filename
imputed_filename= "imputed_"+filename
training_filename="imputed_AqSolDB.csv"
trained_model="svm_solubility_model.pkl"
image_name = filename.split(".")[0] + "_prediction.png"

df = pd.read_csv(filename)
df.head()

In [ ]:
from rdkit.Chem.rdmolops import SanitizeMol
valid_smiles = []
valid_logs = []
for i, row in df.iterrows():
    smi = row['SMILES']
    logS = row['LogS']
    mol = Chem.MolFromSmiles(smi)
    
    if mol is not None:
        try:
            # Sanitize molecule (check valence, aromaticity, conjugation, hybridization)
            SanitizeMol(mol)
            
            # Canonicalize SMILES to have consistent representation
            clean_smi = Chem.MolToSmiles(mol, canonical=True)
            
            valid_smiles.append(clean_smi)
            valid_logs.append(logS)
        
        except Exception as e:
            # Skip molecules that fail sanitization
            continue

# Create a cleaned DataFrame
clean_df = pd.DataFrame({'SMILES': valid_smiles, 'LogS': valid_logs})

print(f"✅ Valid molecules after sanitization: {len(clean_df)} / {len(df)}")
clean_df.head()

In [ ]:
def standardize_molecule(mol):
    try:
        # Remove salts and fragments
        cleaner = rdMolStandardize.Cleanup(mol)
        # Normalize functional groups
        normalizer = rdMolStandardize.Normalize(cleaner)
        # Reionize to standard charge states
        reionizer = rdMolStandardize.Reionize(normalizer)
        return reionizer
    except:
        return mol  # return original if fails

In [ ]:
standardized_smiles = []

for smi in df['SMILES']:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        mol = standardize_molecule(mol)
        clean_smi = Chem.MolToSmiles(mol, canonical=True)
        standardized_smiles.append(clean_smi)
    else:
        standardized_smiles.append(None)

df['Standardized_SMILES'] = standardized_smiles
df = df.dropna(subset=['Standardized_SMILES']).reset_index(drop=True)

df.to_csv(standardized_filename, index=False)
print(f" Standardized dataset saved with {len(df)} molecules")
df.head()

In [ ]:
df = pd.read_csv(standardized_filename)
df.head()

In [ ]:
if 'Unnamed: 0' in df.columns:
    df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
df_reordered = df[['SMILES', 'Standardized_SMILES', 'LogS']]

In [ ]:
calc = Calculator(descriptors, ignore_3D=True)
mols = [Chem.MolFromSmiles(smi) for smi in df["Standardized_SMILES"]]
mols = calc.pandas(mols=mols)

In [ ]:
mols = mols.apply(pd.to_numeric, errors='coerce')
mols.shape
df_descriptors = pd.concat([df_reordered,mols],axis=1)
df_descriptors.shape
df_descriptors.dropna(axis=1,how="all",inplace=True)
df_descriptors.shape
df_descriptors.dropna(axis=0, thresh=len(df_descriptors.columns) * 0.9, inplace=True)
df_descriptors.shape
df_descriptors.dropna(axis=1,how="all",inplace=True)
meta_info = df_reordered.loc[df_descriptors.index]
df_descriptors.drop(columns=["SMILES","Standardized_SMILES","LogS"],inplace=True)
df_descriptors.isna().sum().sum()
imp = SimpleImputer(missing_values=np.nan,strategy="mean")
imputed = imp.fit_transform(df_descriptors)
df_mols_imputed = pd.DataFrame(imputed,columns=df_descriptors.columns,index=df_descriptors.index)
final_df = pd.concat([meta_info, df_mols_imputed], axis=1)
final_df.shape
final_df.to_csv(imputed_filename)

In [ ]:
df_train = pd.read_csv(training_filename)

# Get feature column names (all columns after first 5)
feature_cols = df_train.columns[5:].tolist()

print(f"\n📋 Training model used {len(feature_cols)} features")
df1_descriptors = pd.read_csv(imputed_filename)
# Extract only the features that were used in training

missing = set(feature_cols) - set(df1_descriptors.columns)
print("Missing columns:", missing)

for col in missing:
    df1_descriptors[col] = 0

X1 = df1_descriptors[feature_cols]


# Get actual LogS values
y1_true = df1_descriptors['LogS'].values


print(f"\n✅ Features aligned!")
print(f"   Challenge 1 features: {X1.shape}")



try:
    model_rf = joblib.load(trained_model)
    print("\n✅ Model loaded successfully!")
except FileNotFoundError:
    print("\n❌ Error: Could not find 'random_forest_model.pkl'")
    print("Please make sure you saved your model in the RF training notebook using:")
    print("joblib.dump(model_rf, 'random_forest_model.pkl')")
    raise


y1_pred = model_rf.predict(X1)


print("\n✅ Predictions completed!")
print(f"   Challenge 1: {len(y1_pred)} predictions")


def evaluate_predictions(y_true, y_pred, dataset_name):
    """Calculate and print evaluation metrics"""
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    print(f"\n{'='*50}")
    print(f"📊 {dataset_name} EVALUATION")
    print(f"{'='*50}")
    print(f"R² Score:  {r2:.4f}")
    print(f"RMSE:      {rmse:.4f}")
    print(f"MAE:       {mae:.4f}")
    print(f"{'='*50}")
    
    return r2, rmse, mae

r2_1, rmse_1, mae_1 = evaluate_predictions(y1_true, y1_pred, "Challenge Data 1")


fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(y1_true, y1_pred, alpha=0.6, edgecolors='k', s=30)
min_val = min(y1_true.min(), y1_pred.min())
max_val = max(y1_true.max(), y1_pred.max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
ax.set_xlabel('Actual LogS', fontweight='bold')
ax.set_ylabel('Predicted LogS', fontweight='bold')
ax.set_title(f'Challenge 1: R² = {r2_1:.4f}, MAE = {mae_1:.4f}', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()



In [ ]:
fig.tight_layout()
fig.savefig(image_name, dpi=300, bbox_inches='tight')
plt.show()
print("Figure Saved")

In [ ]:
# (Recommended: use a conda/mamba env with rdkit installed)
import os, json, joblib, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from typing import Tuple, List, Optional
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

SEED = 42
np.random.seed(SEED)

BASE = Path('.')
DATA_DIR = BASE / 'data'
SPLITS_DIR = DATA_DIR / 'splits'
PROC_DIR = DATA_DIR / 'processed'
MODELS_DIR = Path('models')
FIG_DIR = Path('figures')
RESULTS_DIR = Path('results')

for d in [DATA_DIR, SPLITS_DIR, PROC_DIR, MODELS_DIR, FIG_DIR, RESULTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

STD_CSV_CANDIDATES = [
    'standardized_AqSolDB.csv',
    DATA_DIR / 'standardized_AqSolDB.csv',
    'AqSolDB_clean.csv',
    DATA_DIR / 'AqSolDB_clean.csv'
]


In [ ]:

from typing import Sequence

def find_standardized_csv(candidates: Sequence[Path]) -> Path:
    for c in candidates:
        c = Path(c)
        if c.exists():
            return c
    raise FileNotFoundError("Place 'standardized_AqSolDB.csv' next to this notebook or in ./data/")

def guess_columns(df: pd.DataFrame) -> Tuple[str, str]:
    smiles_candidates = [c for c in df.columns if 'smiles' in c.lower()]
    if not smiles_candidates:
        raise ValueError(f"No SMILES-like column found. Columns: {list(df.columns)}")
    smiles_col = smiles_candidates[0]
    y_candidates = [c for c in df.columns if c.lower() in ('logs','log_s','logsol','log_solu','target','y','solubility','aqsol')]
    if not y_candidates:
        numeric_cols = [c for c in df.columns if c != smiles_col and pd.api.types.is_numeric_dtype(df[c])]
        if not numeric_cols:
            raise ValueError("No numeric target column found. Rename your target to 'LogS'.")
        y_col = numeric_cols[0]
    else:
        y_col = y_candidates[0]
    return smiles_col, y_col


In [ ]:

std_path = find_standardized_csv(STD_CSV_CANDIDATES)
print(f"Using standardized data at: {std_path}")
df = pd.read_csv(std_path)
print(df.head(3))
print(f"Rows: {len(df)} | Columns: {list(df.columns)}")

smiles_col, y_col = guess_columns(df)
print(f"Detected SMILES column: {smiles_col} | target column: {y_col}")

id_col = None
for c in ['inchikey', 'InChIKey', 'canonical_smiles', 'can_smiles', 'id']:
    if c in df.columns:
        id_col = c
        break

if id_col is None:
    def to_canonical(s):
        try:
            m = Chem.MolFromSmiles(str(s))
            return Chem.MolToSmiles(m, canonical=True) if m is not None else None
        except Exception:
            return None
    df['canonical_smiles'] = df[smiles_col].apply(to_canonical)
    id_col = 'canonical_smiles'

df = df.dropna(subset=[id_col, y_col]).copy()
before = len(df)
df = df.drop_duplicates(subset=[id_col]).copy()
after = len(df)
print(f"De-duplicated by {id_col}: {before} -> {after}")


In [ ]:

unique_ids = df[id_col].tolist()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=SEED)
train_ids, test_ids = set(train_ids), set(test_ids)

(SPLITS_DIR / 'train_ids.txt').write_text("\n".join(sorted(train_ids)))
(SPLITS_DIR / 'test_ids.txt').write_text("\n".join(sorted(test_ids)))
print("Saved split files:", SPLITS_DIR / 'train_ids.txt', SPLITS_DIR / 'test_ids.txt')

df_train = df[df[id_col].isin(train_ids)].copy()
df_test  = df[df[id_col].isin(test_ids)].copy()
print(f"Train rows: {len(df_train)} | Test rows: {len(df_test)}")
assert set(df_train[id_col]).isdisjoint(set(df_test[id_col]))


In [ ]:

import numpy as np

def maccs_fp(mol: Chem.Mol) -> Optional[np.ndarray]:
    if mol is None:
        return None
    try:
        fp = rdMolDescriptors.GetMACCSKeysFingerprint(mol)
        return np.frombuffer(fp.ToBitString().encode('ascii'), dtype='S1').astype(np.uint8) - ord(b'0')
    except Exception:
        return None

def make_maccs_frame(df_part: pd.DataFrame, smiles_col: str, id_col: str, y_col: str) -> pd.DataFrame:
    recs = []
    for _id, s, y in zip(df_part[id_col], df_part[smiles_col], df_part[y_col]):
        mol = Chem.MolFromSmiles(str(s))
        arr = maccs_fp(mol)
        if arr is None:
            continue
        recs.append({'_id': _id, 'y': float(y), **{f'b{i}': int(arr[i]) for i in range(len(arr))}})
    return pd.DataFrame.from_records(recs)

Xy_train = make_maccs_frame(df_train, smiles_col, id_col, y_col)
Xy_test  = make_maccs_frame(df_test,  smiles_col, id_col, y_col)

print("Train featurized shape:", Xy_train.shape, "Test shape:", Xy_test.shape)
bit_cols = [c for c in Xy_train.columns if c.startswith('b')]
bit_cols.sort(key=lambda z: int(z[1:]))
assert len(bit_cols) == 167, f"Expected 167 MACCS bits, got {len(bit_cols)}"

X_train = Xy_train[bit_cols].to_numpy(dtype=np.uint8)
y_train = Xy_train['y'].to_numpy(dtype=float)
X_test  = Xy_test[bit_cols].to_numpy(dtype=np.uint8)
y_test  = Xy_test['y'].to_numpy(dtype=float)

from pathlib import Path
PROC_DIR = Path('data/processed'); PROC_DIR.mkdir(parents=True, exist_ok=True)
np.save(PROC_DIR / 'maccs_X_train.npy', X_train)
np.save(PROC_DIR / 'maccs_y_train.npy', y_train)
np.save(PROC_DIR / 'maccs_X_test.npy', X_test)
np.save(PROC_DIR / 'maccs_y_test.npy', y_test)


In [ ]:

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
scoring = 'neg_root_mean_squared_error'

rf = RandomForestRegressor(random_state=SEED, n_jobs=-1)
rf_grid = {'n_estimators':[200,500,800], 'max_depth':[None,10,20,40], 'min_samples_leaf':[1,2,4]}

svr = SVR(kernel='rbf')
svr_grid = {'C':[1.0,10.0,100.0,300.0], 'gamma':['scale',0.01,0.03,0.1,0.3], 'epsilon':[0.01,0.05,0.1,0.2]}


In [ ]:

import joblib, json, pandas as pd

rf_search = GridSearchCV(rf, rf_grid, cv=cv, scoring=scoring, n_jobs=-1, verbose=1)
rf_search.fit(X_train, y_train)
print("Best RF params:", rf_search.best_params_)

svr_search = GridSearchCV(svr, svr_grid, cv=cv, scoring=scoring, n_jobs=-1, verbose=1)
svr_search.fit(X_train, y_train)
print("Best SVR params:", svr_search.best_params_)

Path('models').mkdir(exist_ok=True)
joblib.dump(rf_search.best_estimator_, 'models/maccs_rf.joblib')
joblib.dump(svr_search.best_estimator_, 'models/maccs_svr.joblib')

Path('results').mkdir(exist_ok=True)
with open('results/maccs_rf_cv.json','w') as f:
    json.dump({'best_params': rf_search.best_params_, 'best_score_neg_rmse': float(rf_search.best_score_)}, f, indent=2)
with open('results/maccs_svr_cv.json','w') as f:
    json.dump({'best_params': svr_search.best_params_, 'best_score_neg_rmse': float(svr_search.best_score_)}, f, indent=2)


In [ ]:
import numpy as np
import joblib, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score

# RMSE helper: prefer new API, fall back to old
try:
    from sklearn.metrics import root_mean_squared_error as _rmse
    def rmse(y, yhat): return float(_rmse(y, yhat))
except Exception:
    from sklearn.metrics import mean_squared_error
    def rmse(y, yhat): return float(np.sqrt(mean_squared_error(y, yhat)))

def evaluate(name, model, X, y):
    yhat = model.predict(X)
    return {
        'model': name,
        'rmse' : rmse(y, yhat),
        'mae'  : mean_absolute_error(y, yhat),
        'r2'   : r2_score(y, yhat),
        'yhat' : yhat,
    }

rf_best  = joblib.load('models/maccs_rf.joblib')
svr_best = joblib.load('models/maccs_svr.joblib')

res_rf  = evaluate('MACCS-RF',  rf_best,  X_test, y_test)
res_svr = evaluate('MACCS-SVR', svr_best, X_test, y_test)

results_df = pd.DataFrame(
    [{k:v for k,v in res_rf.items()  if k!='yhat'},
     {k:v for k,v in res_svr.items() if k!='yhat'}]
).sort_values('rmse')

results_df.to_csv('results/maccs_test_metrics.csv', index=False)
results_df


In [ ]:

import matplotlib.pyplot as plt, numpy as np, pandas as pd
from pathlib import Path
Path('figures').mkdir(exist_ok=True)

def plot_pred_vs_true(y_true, y_pred, title, savepath):
    plt.figure(figsize=(5,5))
    plt.scatter(y_true, y_pred, alpha=0.5)
    lims = [min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())]
    plt.plot(lims, lims, '--')
    plt.xlabel('True LogS'); plt.ylabel('Predicted LogS'); plt.title(title)
    plt.tight_layout(); plt.savefig(savepath, dpi=200); plt.show()

def plot_residuals(y_true, y_pred, title, savepath):
    resid = y_pred - y_true
    plt.figure(figsize=(6,4))
    plt.hist(resid, bins=30)
    plt.xlabel('Residual (Pred - True)'); plt.ylabel('Count'); plt.title(title)
    plt.tight_layout(); plt.savefig(savepath, dpi=200); plt.show()

plot_pred_vs_true(y_test, res_rf['yhat'],  'MACCS-RF: Pred vs True',  'figures/maccs_rf_pred_vs_true.png')
plot_residuals(y_test, res_rf['yhat'],     'MACCS-RF: Residuals',     'figures/maccs_rf_residuals.png')
plot_pred_vs_true(y_test, res_svr['yhat'], 'MACCS-SVR: Pred vs True', 'figures/maccs_svr_pred_vs_true.png')
plot_residuals(y_test, res_svr['yhat'],    'MACCS-SVR: Residuals',    'figures/maccs_svr_residuals.png')


In [ ]:

# Minimal predict helper
def predict_logs_for_smiles(smiles_list: List[str], model_name='rf'):
    import joblib, numpy as np
    path = 'models/maccs_rf.joblib' if model_name.lower()=='rf' else 'models/maccs_svr.joblib'
    model = joblib.load(path)
    mats = []
    for s in smiles_list:
        mol = Chem.MolFromSmiles(s)
        if mol is None:
            raise ValueError(f"Invalid SMILES: {s}")
        fp = rdMolDescriptors.GetMACCSKeysFingerprint(mol)
        arr = np.frombuffer(fp.ToBitString().encode('ascii'), dtype='S1').astype(np.uint8) - ord(b'0')
        mats.append(arr)
    X_new = np.vstack(mats)
    return model.predict(X_new)

# Example:
# predict_logs_for_smiles(['CCO','c1ccccc1O'], 'rf')


In [ ]:

manifest = {
    'seed': SEED,
    'notes': 'Self-created 80/20 split for MACCS pipelines',
    'files': {
        'split_train': 'data/splits/train_ids.txt',
        'split_test': 'data/splits/test_ids.txt',
        'metrics_csv': 'results/maccs_test_metrics.csv',
        'rf_model': 'models/maccs_rf.joblib',
        'svr_model': 'models/maccs_svr.joblib',
        'figures': [
            'figures/maccs_rf_pred_vs_true.png',
            'figures/maccs_rf_residuals.png',
            'figures/maccs_svr_pred_vs_true.png',
            'figures/maccs_svr_residuals.png'
        ]
    }
}
with open('results/maccs_run_manifest.json','w') as f:
    json.dump(manifest, f, indent=2)
print('Saved manifest → results/maccs_run_manifest.json')


In [ ]:
# ---- config: point to your challenge files ----
CHALLENGE_FILES = [
    "challenge_data_1.csv",
    "challenge_data_2.csv",
]

# ---- imports ----
import os, json, numpy as np, pandas as pd, joblib
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from sklearn.metrics import mean_absolute_error, r2_score

# RMSE helper (works on old/new scikit-learn)
try:
    from sklearn.metrics import root_mean_squared_error as _rmse
    def rmse(y, yhat): return float(_rmse(y, yhat))
except Exception:
    from sklearn.metrics import mean_squared_error
    def rmse(y, yhat): return float(np.sqrt(mean_squared_error(y, yhat)))

Path("results").mkdir(exist_ok=True)

# ---- small helpers ----
TARGET_NAMES = {"logs","log_s","logsol","log_solu","target","y","solubility","aqsol"}

def detect_cols(df: pd.DataFrame):
    smiles_cols = [c for c in df.columns if "smiles" in c.lower()]
    if not smiles_cols:
        raise ValueError(f"No SMILES-like column found. Columns: {list(df.columns)}")
    smiles_col = smiles_cols[0]
    y_col = next((c for c in df.columns if c.lower() in TARGET_NAMES), None)
    return smiles_col, y_col

def canonicalize(s: str):
    m = Chem.MolFromSmiles(str(s))
    return Chem.MolToSmiles(m, canonical=True) if m is not None else None

def maccs_matrix(smiles_list):
    X = []
    kept = []
    for s in smiles_list:
        m = Chem.MolFromSmiles(s)
        if m is None: 
            continue
        fp = rdMolDescriptors.GetMACCSKeysFingerprint(m)
        arr = np.frombuffer(fp.ToBitString().encode("ascii"), dtype="S1").astype(np.uint8) - ord(b"0")
        X.append(arr); kept.append(s)
    if not X:
        raise RuntimeError("No valid molecules after featurization.")
    X = np.vstack(X)
    assert X.shape[1] == 167, f"Expected 167 MACCS bits, got {X.shape[1]}"
    return X, kept

# ---- load saved models ----
rf  = joblib.load("models/maccs_rf.joblib")
svr = joblib.load("models/maccs_svr.joblib")

def evaluate_challenge(csv_path: str):
    df_raw = pd.read_csv(csv_path)
    smiles_col, y_col = detect_cols(df_raw)

    # minimal cleaning to match training assumptions
    df_raw["canonical_smiles"] = df_raw[smiles_col].map(canonicalize)
    df = df_raw.dropna(subset=["canonical_smiles"]).drop_duplicates("canonical_smiles").copy()

    # features
    X, ids = maccs_matrix(df["canonical_smiles"].tolist())

    # predict
    pred_rf  = rf.predict(X)
    pred_svr = svr.predict(X)

    out_rf  = pd.DataFrame({"canonical_smiles": ids, "y_pred": pred_rf})
    out_svr = pd.DataFrame({"canonical_smiles": ids, "y_pred": pred_svr})

    metrics = {}
    if y_col is not None and pd.api.types.is_numeric_dtype(df[y_col]):
        y_true = df[y_col].astype(float).to_numpy()[:len(ids)]
        metrics = {
            "RF" : {"rmse": rmse(y_true, pred_rf),  "mae": float(np.mean(np.abs(y_true - pred_rf))),  "r2": float(r2_score(y_true, pred_rf))},
            "SVR": {"rmse": rmse(y_true, pred_svr), "mae": float(np.mean(np.abs(y_true - pred_svr))), "r2": float(r2_score(y_true, pred_svr))},
        }

    # save
    stem = Path(csv_path).stem
    out_rf.to_csv(f"results/{stem}_predictions_rf.csv", index=False)
    out_svr.to_csv(f"results/{stem}_predictions_svr.csv", index=False)
    if metrics:
        with open(f"results/{stem}_metrics_maccs.json", "w") as f:
            json.dump(metrics, f, indent=2)

    return metrics, out_rf.head(), out_svr.head()

# ---- run on all challenge files ----
for f in CHALLENGE_FILES:
    print(f"\n=== {f} ===")
    m, head_rf, head_svr = evaluate_challenge(f)
    if m:
        print(json.dumps(m, indent=2))
    else:
        print("No ground truth column found; saved predictions only.")
    display(head_rf, head_svr)
